Starting with face recognition and basics to see if it works.

In [1]:
# Importing all the libraries in one go.

import numpy as np
import cv2
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.models import model_from_json
from os import listdir
import os
import tensorflow

Using TensorFlow backend.


In [33]:
face_cascade = cv2.CascadeClassifier('C:\ProgramData\Anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')


In [34]:
face_cascade.load('C:\ProgramData\Anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')

True

In [35]:
# Setting the present directory where all the files are available.
os.chdir("D:\Learning/Machine Learning Training\PGP ML\Capstone Project\\")
dir_path = "D:\Learning/Machine Learning Training\PGP ML\Capstone Project\\"

In [5]:
os.path.exists(dir_path)

True

In [36]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(160, 160))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    #preprocess_input normalizes input in scale of [-1, +1]. You must apply same normalization in prediction.
    #Ref: https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py (Line 45)
    img = preprocess_input(img)
    return img

In [7]:
from inception_resnet_v1 import *
model = InceptionResNetV1()

In [37]:
model.load_weights('facenet_weights.h5')
print("weights loaded")

weights loaded


In [38]:
# Finding Eucledian distance between 2 images
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [39]:
threshold = 21 #tuned threshold for l2 disabled euclidean distance

In [40]:
employee_pictures = "D:\Learning/Machine Learning Training\PGP ML\Capstone Project\database\\"

In [41]:
employees = dict()

In [42]:
for file in listdir(employee_pictures):
	employee, extension = file.split(".")
	img = preprocess_image('D:\Learning/Machine Learning Training\PGP ML\Capstone Project\database\\%s.jpeg' % (employee))
	representation = model.predict(img)[0,:]
	
	employees[employee] = representation
	
print("employee representations retrieved successfully")

employee representations retrieved successfully


In [43]:
cap = cv2.VideoCapture(0) #webcam

In [15]:
#ret, img = cap.read()

In [16]:
 #cap.isOpened()

True

In [17]:
#print(img)

In [44]:
while(True):
  ret, img = cap.read()
  #print(ret,img)
  faces = face_cascade.detectMultiScale(img, 1.3, 5)
  #faces = face_cascade.detectMultiScale('/content/drive/My Drive/Colab Notebooks/Capstone Project/new one/screenshot/photo.jpg', 1.3, 5)
	
  for (x,y,w,h) in faces:
    if w > 130: #discard small detected faces
      cv2.rectangle(img, (x,y), (x+w,y+h), (67, 67, 67), 1) #draw rectangle to main image
			
      detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
      detected_face = cv2.resize(detected_face, (160, 160)) #resize to 224x224
			
      img_pixels = image.img_to_array(detected_face)
      img_pixels = np.expand_dims(img_pixels, axis = 0)
			#employee dictionary is using preprocess_image and it normalizes in scale of [-1, +1]
      img_pixels /= 127.5
      img_pixels -= 1
			
      captured_representation = model.predict(img_pixels)[0,:]
			
      distances = []
			
      for i in employees:
        employee_name = i
        source_representation = employees[i]
				
        distance = findEuclideanDistance(captured_representation, source_representation)
				
				#print(employee_name,": ",distance)
        distances.append(distance)
			
      label_name = 'unknown'
      index = 0
      for i in employees:
        employee_name = i
        if index == np.argmin(distances):
          if distances[index] <= threshold:
						#print("detected: ",employee_name)
						
						#label_name = "%s (distance: %s)" % (employee_name, str(round(distance,2)))
            similarity = 100 + (20 - distance)
            if similarity > 99.99: similarity = 99.99
						
            label_name = "%s (%s%s)" % (employee_name, str(round(similarity,2)), '%')
						
            break
				#	
        index = index + 1
			
      cv2.putText(img, label_name, (int(x+w+15), int(y-64)), cv2.FONT_HERSHEY_SIMPLEX, 1, (67,67,67), 2)
					
			#connect face and text
      cv2.line(img,(x+w, y-64),(x+w-25, y-64),(67,67,67),1)
      cv2.line(img,(int(x+w/2),y),(x+w-25,y-64),(67,67,67),1)
			
  cv2.imshow('img',img)
	
  if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
    break

In [32]:
#kill open cv thingsq
cap.release()
cv2.destroyAllWindows()